# Causal Web - Analysis Notebook
This notebook is designed to analyze the JSON log files generated by the CWT simulation.

In [ ]:
# ## 1. Setup & Imports
import json
import pandas as pd
import networkx as nx
from pathlib import Path
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

print("Libraries imported successfully.")

## 2. Configuration & Data Loading
Define the path to your log directory and load all log files into pandas DataFrames.

In [ ]:
LOG_DIR = Path('./logs')

def load_log_file(log_path: Path) -> pd.DataFrame:
    """Safely loads a JSON log file into a pandas DataFrame."""
    if not log_path.exists():
        print(f'⚠️ Warning: Log file not found at {log_path}. Returning empty DataFrame.')
        return pd.DataFrame()
    try:
        df = pd.read_json(log_path, lines=True)
        print(f'✅ Successfully loaded {log_path.name} with {len(df)} records.')
        return df
    except ValueError:
        print(f'❌ Error: Could not decode JSON from {log_path}. Is it a valid JSONL file?')
        return pd.DataFrame()

emergence_log_df = load_log_file(LOG_DIR / 'node_emergence_log.json')
structural_growth_df = load_log_file(LOG_DIR / 'structural_growth_log.json')
collapse_chain_df = load_log_file(LOG_DIR / 'collapse_chain_log.json')
law_wave_log_df = load_log_file(LOG_DIR / 'law_wave_log.json')
failure_log_df = load_log_file(LOG_DIR / 'propagation_failure_log.json')
stable_freq_df = load_log_file(LOG_DIR / 'stable_frequency_log.json')
classical_map_df = load_log_file(LOG_DIR / 'classicalization_map.json')
node_state_df = load_log_file(LOG_DIR / 'node_state_log.json')
coherence_log_df = load_log_file(LOG_DIR / 'coherence_log.json')
cluster_log_df = load_log_file(LOG_DIR / 'cluster_log.json')

## 3. Overall Simulation Health Dashboard 📈
High-level metrics and visualizations for the entire simulation run.

In [ ]:
if not structural_growth_df.empty:
    fig_growth = go.Figure()
    fig_growth.add_trace(go.Scatter(x=structural_growth_df['tick'], y=structural_growth_df['node_count'], mode='lines', name='Total Nodes'))
    fig_growth.add_trace(go.Scatter(x=structural_growth_df['tick'], y=structural_growth_df['edge_count'], mode='lines', name='Total Edges', line=dict(color='orange')))
    fig_growth.update_layout(title='Node and Edge Growth Over Time', xaxis_title='Tick', yaxis_title='Count')
    fig_growth.show()

if not emergence_log_df.empty:
    propagation_counts = emergence_log_df['mechanism'].value_counts()
    fig_pie = px.pie(values=propagation_counts.values, names=propagation_counts.index, title='Propagation Mechanism Ratio (SIP vs. CSP)', color_discrete_map={'SIP': 'royalblue', 'CSP': 'firebrick'})
    fig_pie.show()

if not structural_growth_df.empty and 'avg_coherence' in structural_growth_df.columns:
    fig_coherence = go.Figure()
    fig_coherence.add_trace(go.Scatter(x=structural_growth_df['tick'], y=structural_growth_df['avg_coherence'], mode='lines', name='Average Coherence', line=dict(color='green')))
    if 'avg_decoherence_debt' in structural_growth_df.columns:
        fig_coherence.add_trace(go.Scatter(x=structural_growth_df['tick'], y=structural_growth_df['avg_decoherence_debt'], mode='lines', name='Average Decoherence Debt', line=dict(color='red')))
    fig_coherence.update_layout(title='Global Coherence and Decoherence Over Time', xaxis_title='Tick', yaxis_title='Value')
    fig_coherence.show()

## 4. Emergence & Evolution Analysis 🧠

In [ ]:
if not law_wave_log_df.empty:
    final_law_waves = law_wave_log_df.loc[law_wave_log_df.groupby('cluster_id')['tick'].idxmax()]
    dominant_frequencies = final_law_waves['stable_frequency'].value_counts()
    fig_lw = px.bar(x=dominant_frequencies.index.astype(str), y=dominant_frequencies.values, title='Dominant Law-Wave Frequencies at End of Simulation', labels={'x': 'Stable Frequency (Hz)', 'y': 'Number of Clusters'})
    fig_lw.show()

if not emergence_log_df.empty and not collapse_chain_df.empty:
    csp_events = emergence_log_df[emergence_log_df['mechanism'] == 'CSP']
    collapse_events = collapse_chain_df.drop_duplicates(subset=['chain_id'])
    fig_timeline = go.Figure()
    fig_timeline.add_trace(go.Scatter(x=collapse_events['tick'], y=[1] * len(collapse_events), mode='markers', marker=dict(symbol='x', color='red', size=15), name='Major Collapse Event', hovertext=collapse_events['chain_id']))
    fig_timeline.add_trace(go.Scatter(x=csp_events['tick'], y=[1.05] * len(csp_events), mode='markers', marker=dict(symbol='circle', color='royalblue', size=10), name='CSP Node Generation', hovertext=csp_events['node_id']))
    fig_timeline.update_layout(title='Timeline of Collapse-Seeded Propagation (CSP) Events', xaxis_title='Tick', yaxis=dict(showticklabels=False, range=[0.9, 1.2]), showlegend=True)
    fig_timeline.show()

## 4a. Law-Wave Speciation and Dominance 🔬
This plot shows how many distinct stable frequencies exist at each tick.

In [ ]:

if not stable_freq_df.empty:
    diversity_records = []
    for rec in stable_freq_df.itertuples(index=False):
        tick, data = list(rec._asdict().items())[0]
        tick = int(tick)
        unique = len(set(data.values()))
        diversity_records.append({'tick': tick, 'diversity': unique})
    df_div = pd.DataFrame(diversity_records).sort_values('tick')
    fig = px.line(df_div, x='tick', y='diversity', title='Stable Frequency Diversity Over Time', labels={'diversity':'Unique Frequencies'})
    fig.show()
else:
    print('stable_frequency_log.json not found or empty')


## 4b. The Classicalization Front 🌍
Visualise how coherent regions collapse over time.

In [ ]:

if not classical_map_df.empty and not node_state_df.empty:
    with open(LOG_DIR / 'runtime_graph_snapshots/graph_0.json') as f:
        base_graph = json.load(f)
    positions = {n['id']:(n['x'], n['y']) for n in base_graph['nodes']}
    sample_ticks = sorted(classical_map_df.columns[:5]) if len(classical_map_df.columns)>0 else []
    for tick in sample_ticks:
        state = classical_map_df[str(tick)] if str(tick) in classical_map_df else {}
        debt = node_state_df.get(str(tick), {}).get('debt', {}) if isinstance(node_state_df, dict) else node_state_df.loc[str(tick),'debt'] if str(tick) in node_state_df.index else {}
        data = []
        for nid, pos in positions.items():
            if str(tick) in classical_map_df and nid in classical_map_df[str(tick)]:
                collapsed = classical_map_df[str(tick)][nid]
            else:
                collapsed = False
            d = debt.get(nid, 0) if isinstance(debt, dict) else (debt[nid] if nid in debt else 0)
            if collapsed:
                color = 'red'
            elif d > 0.5:
                color = 'yellow'
            else:
                color = 'blue'
            data.append({'x':pos[0],'y':pos[1],'color':color,'id':nid})
        df = pd.DataFrame(data)
        fig = px.scatter(df, x='x', y='y', color='color', title=f'Classicalization Front at tick {tick}', color_discrete_map={'blue':'blue','yellow':'yellow','red':'red'})
        fig.update_traces(marker=dict(size=12))
        fig.show()
else:
    print('Required logs for classicalization front not available')


## 4c. The Creative Cycle (SIP vs. CSP)

In [ ]:

if not coherence_log_df.empty and not structural_growth_df.empty:
    avg_coh = []
    for rec in coherence_log_df.itertuples(index=False):
        tick, values = list(rec._asdict().items())[0]
        tick = int(tick)
        avg = sum(values.values())/len(values)
        avg_coh.append({'tick':tick,'coherence':avg})
    coh_df = pd.DataFrame(avg_coh).sort_values('tick')
    sg = structural_growth_df
    sg['ratio'] = sg.apply(lambda r: (r['sip_success'] or 0)/(r['csp_success'] or 1), axis=1)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=coh_df['tick'], y=coh_df['coherence'], name='Avg Coherence'), secondary_y=False)
    fig.add_trace(go.Scatter(x=sg['tick'], y=sg['ratio'], name='SIP/CSP Ratio', line=dict(color='orange')), secondary_y=True)
    fig.update_layout(title='Creative Cycle Dynamics')
    fig.update_yaxes(title_text='Average Coherence', secondary_y=False)
    fig.update_yaxes(title_text='SIP/CSP Ratio', secondary_y=True)
    fig.show()
else:
    print('coherence_log or structural_growth_log not available')


## 4d. Causal Genetics and Lineage Tracing 🧬

In [ ]:

if not stable_freq_df.empty and not emergence_log_df.empty:
    latest_tick = max(int(t) for t in stable_freq_df.columns)
    stable_nodes = stable_freq_df[str(latest_tick)].keys()
    parent_map = {row['id']: row['parents'] for _, row in emergence_log_df.iterrows()}
    def lineage(nid):
        chain = [nid]
        while nid in parent_map and parent_map[nid]:
            nid = parent_map[nid][0]
            chain.append(nid)
        return ' -> '.join(chain)
    for nid in list(stable_nodes)[:5]:
        print(f"Lineage for {nid}: {lineage(nid)}")
else:
    print('Insufficient data for lineage tracing')


## 5. Causal Chain Forensics 🕵️

In [ ]:
def visualize_collapse_chain(chain_id: str):
    """Draws a graph of a specific collapse chain."""
    if collapse_chain_df.empty:
        print('Collapse log is empty.')
        return
    chain_data = collapse_chain_df[collapse_chain_df['chain_id'] == chain_id]
    if chain_data.empty:
        print(f'No data found for chain_id {chain_id!r}.')
        return
    G = nx.from_pandas_edgelist(chain_data, 'source', 'target', create_using=nx.DiGraph())
    if not G.nodes:
        print('Graph could not be created from the chain data.')
        return
    pos = nx.spring_layout(G, seed=42)
    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.5, color='#888'), hoverinfo='none', mode='lines')
    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]
    node_adjacencies = [len(list(G.successors(node))) + len(list(G.predecessors(node))) for node in G.nodes()]
    node_text = [f'Node: {node}<br># of connections: {adj}' for node, adj in zip(G.nodes(), node_adjacencies)]
    node_trace = go.Scatter(x=node_x, y=node_y, mode='markers+text', hoverinfo='text', text=[str(node) for node in G.nodes()], textposition='top center', marker=dict(showscale=True, colorscale='YlGnBu', reversescale=True, color=node_adjacencies, size=15, colorbar=dict(thickness=15, title='Node Connections', xanchor='left', titleside='right'), line_width=2))
    node_trace.hovertext = node_text
    fig = go.Figure(data=[edge_trace, node_trace], layout=go.Layout(title=f'Collapse Chain Visualization for Chain ID: {chain_id}', titlefont_size=16, showlegend=False, hovermode='closest', margin=dict(b=20,l=5,r=5,t=40), annotations=[dict(text='Graph of how a collapse propagated through entangled nodes.', showarrow=False, xref='paper', yref='paper', x=0.005, y=-0.002)], xaxis=dict(showgrid=False, zeroline=False, showticklabels=False), yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
    fig.show()

if not collapse_chain_df.empty:
    example_chain_id = collapse_chain_df['chain_id'].iloc[0]
    print(f'
--- Visualizing example collapse chain: {example_chain_id} ---')
    visualize_collapse_chain(example_chain_id)
else:
    print('
Could not run visualization example because collapse log is empty.')